<a href="https://colab.research.google.com/github/marina554/Consolidated-Accounting/blob/main/Consolidated_Balance_Sheet_and_Cash_Flow_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

# =========================================
# Simple Consolidated Model (Parent & 1 Child)
# =========================================

# ---------------------------
# 1. Individual company financials
# ---------------------------
parent_accounts = {"Cash": 1000, "Accounts_Receivable": 500, "Revenue": 2000, "Cost_of_Goods_Sold": 800, "Intercompany_Receivable": 200}
child_accounts = {"Cash": 300, "Accounts_Receivable": 100, "Revenue": 500, "Cost_of_Goods_Sold": 200, "Intercompany_Payable": 200}
ownership = 0.6  # Parent owns 60% of child

# ---------------------------
# 2. Internal transaction elimination
# ---------------------------
def eliminate_internal_transaction(parent, child, parent_key, child_key):
    """
    Eliminate intercompany transaction between parent and child
    """
    eliminated_amount = min(parent[parent_key], child[child_key])
    parent[parent_key] -= eliminated_amount
    child[child_key] -= eliminated_amount
    return eliminated_amount

# Eliminate intercompany transaction
eliminated = eliminate_internal_transaction(parent_accounts, child_accounts, "Intercompany_Receivable", "Intercompany_Payable")
print(f"Eliminated intercompany transaction: {eliminated}")

# ---------------------------
# 3. Create consolidated balance sheet
# ---------------------------
def create_consolidated_bs(parent, child, ownership):
    consolidated = {}
    for key in parent.keys():
        if key not in child:
            consolidated[key] = parent[key]
        else:
            consolidated[key] = parent[key] + child[key]
    # Non-controlling interest (NCI)
    nci = {key: child[key] * (1 - ownership) for key in child.keys()}
    return consolidated, nci

consolidated_bs, nci = create_consolidated_bs(parent_accounts, child_accounts, ownership)
df_consolidated = pd.DataFrame([consolidated_bs, nci], index=["Consolidated", "Non-controlling Interest"])
print("\n=== Consolidated Balance Sheet ===")
print(df_consolidated)

# ---------------------------
# 4. Simple cash flow calculation
# ---------------------------
def simple_cashflow(parent, child):
    """
    Simple cash flow calculation (direct method, operating CF only)
    """
    total_cash_start = parent["Cash"] + child["Cash"]

    # Operating CF = Revenue - COGS (simplified)
    parent_cf = parent["Revenue"] - parent["Cost_of_Goods_Sold"]
    child_cf = child["Revenue"] - child["Cost_of_Goods_Sold"]
    total_cf = parent_cf + child_cf

    cash_end = total_cash_start + total_cf

    return pd.DataFrame({
        "Beginning_Cash": [total_cash_start],
        "Operating_CF": [total_cf],
        "Ending_Cash": [cash_end]
    }, index=["Consolidated"])

df_cf = simple_cashflow(parent_accounts, child_accounts)
print("\n=== Consolidated Cash Flow (Simple) ===")
print(df_cf)


Eliminated intercompany transaction: 200

=== Consolidated Balance Sheet ===
                            Cash  Accounts_Receivable  Revenue  \
Consolidated              1300.0                600.0   2500.0   
Non-controlling Interest   120.0                 40.0    200.0   

                          Cost_of_Goods_Sold  Intercompany_Receivable  \
Consolidated                          1000.0                      0.0   
Non-controlling Interest                80.0                      NaN   

                          Intercompany_Payable  
Consolidated                               NaN  
Non-controlling Interest                   0.0  

=== Consolidated Cash Flow (Simple) ===
              Beginning_Cash  Operating_CF  Ending_Cash
Consolidated            1300          1500         2800


In [1]:
import pandas as pd

# 単体財務データ（親会社・子会社）
parent_accounts = {"現金":1000,"売掛金":500,"売上高":2000,"仕入":800,"親子間売掛金":200}
child_accounts = {"現金":300,"売掛金":100,"売上高":500,"仕入":200,"親子間買掛金":200}
ownership = 0.6  # 親会社の持分比率

# 内部取引消去関数
def eliminate_internal_transaction(parent, child, parent_key, child_key):
    eliminated_amount = min(parent[parent_key], child[child_key])
    parent[parent_key] -= eliminated_amount
    child[child_key] -= eliminated_amount
    return eliminated_amount

# 内部取引消去
eliminate_internal_transaction(parent_accounts, child_accounts, "親子間売掛金", "親子間買掛金")

# 連結B/S作成
def create_consolidated_bs(parent, child, ownership):
    consolidated = {}
    for key in parent.keys():
        if key not in child:
            consolidated[key] = parent[key]
        else:
            consolidated[key] = parent[key] + child[key]
    # 非支配株主持分（子会社残高の40%）
    non_controlling_interest = {key: child[key]*(1-ownership) for key in child.keys()}
    return consolidated, non_controlling_interest

consolidated_bs, nci = create_consolidated_bs(parent_accounts, child_accounts, ownership)
df_consolidated = pd.DataFrame([consolidated_bs, nci], index=["連結","非支配株主持分"])
print("=== 連結B/S ===")
print(df_consolidated)

# -----------------------------
# 簡易キャッシュフロー計算
# -----------------------------
def simple_cashflow(parent, child):
    """
    営業CFだけ簡易計算
    """
    # 親子会社の現金を合算
    total_cash_start = parent["現金"] + child["現金"]

    # 営業CF（売上-仕入を現金ベースと仮定）
    # 内部取引はすでに消去済みなのでそのまま計算可能
    parent_cf = parent["売上高"] - parent["仕入"]
    child_cf = child["売上高"] - child["仕入"]

    total_cf = parent_cf + child_cf

    # 現金増減（簡易）
    cash_end = total_cash_start + total_cf

    return pd.DataFrame({
        "現金期首": [total_cash_start],
        "営業CF": [total_cf],
        "現金期末": [cash_end]
    }, index=["連結"])

df_cf = simple_cashflow(parent_accounts, child_accounts)
print("\n=== 連結キャッシュフロー（簡易） ===")
print(df_cf)


=== 連結B/S ===
             現金    売掛金     売上高      仕入  親子間売掛金  親子間買掛金
連結       1300.0  600.0  2500.0  1000.0     0.0     NaN
非支配株主持分   120.0   40.0   200.0    80.0     NaN     0.0

=== 連結キャッシュフロー（簡易） ===
    現金期首  営業CF  現金期末
連結  1300  1500  2800
